# EDA for promoted_content.csv

In [44]:
from pyspark.sql.types import *
import pyspark.sql.functions as F
import pandas as pd
import seaborn as sns
%matplotlib inline

In [45]:
# bucket 설정

mangodm_bucket = "gs://upload-bigquery180927/"

In [46]:
# promoted_contetn 스키마 설정

promoted_content_schema = StructType(
                    [StructField("ad_id", IntegerType(), True),
                    StructField("document_id", IntegerType(), True),                    
                    StructField("campaign_id", IntegerType(), True),
                    StructField("advertiser_id", IntegerType(), True)]
                    )

# google storage에 저장된 promoted_content.csv 파일 로드

promoted_content_df = spark.read.schema(promoted_content_schema).options(header='true', inferschema='false', nullValue='\\N') \
                .csv(mangodm_bucket + "promoted_content.csv")

In [47]:
# spark의 dataframe을 pandas의 dataframe으로 변환

promoted = promoted_content_df.toPandas()

## 1. promoted_content.csv 구조 파악

### 1.1 테이블 구조 및 결측치 확인
- promoted_content.csv는 약 56만개의 행과 4개의 변수로 이루어져 있다.
    - **advertiser_id** : 광고주
    - **campaign_id** : 광고주가 설정한 캠페인
    - **document_id** : 랜딩페이지 즉, 광고주가 광고하기 원하는 콘텐츠
    - **ad_id** : 개별 광고 즉, 랜딩페이지를 광고하는 광고들이다. 
- 4개의 변수 모두 결측치가 하나도 없으며, 자료형도 모두 **정수형(int64)**이다. 하지만 각 숫자가 의미를 갖는 게 아니므로 **실질적으로는 모두 범주형 변수**이며, 이후에 범주형 변수로 변환할 필요가 있다.

In [48]:
promoted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 559583 entries, 0 to 559582
Data columns (total 4 columns):
ad_id            559583 non-null int64
document_id      559583 non-null int64
campaign_id      559583 non-null int64
advertiser_id    559583 non-null int64
dtypes: int64(4)
memory usage: 17.1 MB


In [49]:
promoted.isnull().sum()

ad_id            0
document_id      0
campaign_id      0
advertiser_id    0
dtype: int64

In [50]:
promoted.head()

,ad_id,document_id,campaign_id,advertiser_id
0,1,6614,1,7
1,2,471467,2,7
2,3,7692,3,7
3,4,471471,2,7
4,5,471472,2,7


In [51]:
promoted.tail()

,ad_id,document_id,campaign_id,advertiser_id
559578,572968,1375496,18617,3223
559579,573016,1051466,18617,3223
559580,573081,497486,18617,3223
559581,573094,2868067,18906,283
559582,573098,2790678,13933,16


### 1.2 고유값 개수 확인
- 총 559,583개의 행 중 각 변수별 고유한 행의 개수는 아래와 같다.
- 데이터의 구조와 의미를 보건대, 광고가 각 랜딩페이지에 속하고, 랜딩페이지가 캠페인에 속하며, 캠페인이 광고주에 속하는 **계층적 구조**인 것으로 파악된다.

In [52]:
promoted.nunique()

ad_id            559583
document_id      185709
campaign_id       34675
advertiser_id      4385
dtype: int64

## 2. advertiser_id(광고주)
- 총 4,385명의 고유한 광고주들이 있다.

In [53]:
promoted['advertiser_id'].nunique()

4385

광고주로 묶어서(groupby) 광고주별 고유한 캠페인, 랜딩페이지, 광고의 개수(unique row)를 확인했다.

예를 들어, advertiser_id가 4번인 광고주는 
    - 32개의 캠페인을 운용하고 있고 
    - 80개의 랜딩페이지를 
    - 168개의 광고를 통해 광고하고 있다.

In [54]:
# advertiser_id로 groupby해서 ad_id, campaign_id, document_id의 고유한 값 개수를 확인

grouped_advertiser = promoted.groupby('advertiser_id') \
                            .agg({'campaign_id' : 'nunique', 
                                  'document_id' : 'nunique', 
                                  'ad_id' : 'nunique'}) \
                            .reset_index()
grouped_advertiser.head()

,advertiser_id,ad_id,campaign_id,document_id
0,2,2,1,1
1,3,12,1,2
2,4,168,32,80
3,5,45,2,30
4,6,15,2,8


In [91]:
# 광고주에 관한 기초통계량
# 각 광고주는 평균적으로 8개의 캠페인과 43개의 랜딩페이지, 128개의 광고를 갖고 있다.

grouped_advertiser.describe().round().astype(int) # 기초통계량을 반올림해서 정수형으로 만듦.

,advertiser_id,ad_id,campaign_id,document_id
count,4385,4385,4385,4385
mean,2247,128,8,43
std,1297,684,29,299
min,2,1,1,1
25%,1122,2,1,1
50%,2247,8,1,3
75%,3371,36,4,10
max,4532,16529,448,11637


Q. issue 1
- 1.2에서 document_id의 고유값 개수를 확인했을 땐, 185,709개였다. 하지만, advertiser_id로 groupby해서 document_id의 고유값 개수를 확인해보니 188,048개로 나타났다.
- 아마 document_id가 하나의 advertiser_id에만 속하지 않고 여러 개의 advertiser_id에 속하는 것으로 보인다. 
- 다른 변수들의 고유값 개수는 모두 1.2에서 확인했을 때와 일치하는 것으로 보아 document_id에만 이런 문제가 발생하는 것으로 보인다.


In [67]:
grouped_advertiser.sum()

advertiser_id    9853090
ad_id             559583
campaign_id        34675
document_id       188048
dtype: int64

## 3. campaign_id(캠페인)
- 총 34,675개의 고유한 캠페인이 있다.

In [74]:
promoted['campaign_id'].nunique()

34675

캠페인으로 묶어서 캠페인별 고유한 광고주, 랜딩페이지, 광고의 개수(unique row)를 확인했다.

예를 들어, ID가 7243인 캠페인은
    - 하나의 광고주와
    - 3,806개의 랜딩페이지들,
    - 3,806개의 광고들이 속해 있다.

In [ ]:
Q. issue 2
- 하나의 캠페인이 

In [75]:
grouped_campaign = promoted.groupby('campaign_id') \
                            .agg({'advertiser_id' : 'nunique',
                                 'document_id' : 'nunique',
                                 'ad_id' : 'nunique'}) \
                            .sort_values(['advertiser_id', 'document_id', 'ad_id'], ascending = False) \
                            .reset_index()
grouped_campaign.head(10)

,campaign_id,advertiser_id,ad_id,document_id
0,7243,1,3806,3806
1,16904,1,3604,3604
2,13578,1,3049,3049
3,16622,1,2765,2765
4,8079,1,2738,2738
5,5125,1,2519,2516
6,462,1,1906,1906
7,27171,1,1766,1766
8,933,1,1751,1751
9,27168,1,1715,1715


In [76]:
grouped_campaign.sum()

campaign_id      606935443
advertiser_id        34675
ad_id               559583
document_id         237838
dtype: int64

## 4. document_id(랜딩페이지)
- 총 185,709개의 고유한 랜딩페이지가 있다.

In [77]:
promoted['document_id'].nunique()

185709

위의 issue 1을 확인하기 위해 랜딩페이지로 묶어서 나머지 변수들의 고유값 개수를 확인해 보았다.
- 하나의 랜딩페이지가 여러 개의 광고주에 속한 경우가 있었다. 이를 해석하자면, **여러 명의 광고주가 하나의 랜딩페이지를 광고하고 있다**는 의미이다.

In [66]:
# docuemnt_id로 groupby해서 나머지 변수들의 고유값 개수를 확인.

grouped_document = promoted.groupby('document_id') \
                            .agg({'campaign_id' : 'nunique', 
                                  'advertiser_id' : 'nunique', 
                                  'ad_id' : 'nunique'}) \
                            .sort_values(['advertiser_id', 'ad_id', 'campaign_id'], ascending = False) \
                            .reset_index()
grouped_document.head(10)

,document_id,advertiser_id,ad_id,campaign_id
0,602076,5,529,79
1,469489,5,502,120
2,500812,5,219,52
3,786955,5,63,18
4,1457978,5,57,14
5,1463156,5,45,9
6,1155131,5,34,8
7,985949,4,251,44
8,257734,4,214,44
9,363434,4,201,18


아래의 방법으로 확인해보니 완벽한 계층구조가 아님을 더욱 확실히 알 수 있다.
- 하나의 랜딩페이지가 여러 개의 캠페인에 속해 있다.

In [70]:
grouped_document.sum()

document_id      298511424050
advertiser_id          188048
ad_id                  559583
campaign_id            237838
dtype: int64

## 5. ad_id(광고)
- 총 559,583개의 고유한 광고가 있다.

In [78]:
promoted['ad_id'].nunique()

559583

광고로 묶어서 광고별 고유한 광고주, 캠페인, 랜딩페이지의 개수(unique row)를 확인했다.

데이터의 행 개수와 ad_id의 행 개수가 일치하기 때문에 당연한 말이지만,

확인해 보니 하나의 광고는
    - 하나의 랜딩페이지와
    - 하나의 캠페인,
    - 하나의 광고주에 속해 있었다.

In [80]:
# ad_id로 groupby해서 나머지 변수들의 고유값 개수를 확인.

grouped_ad = promoted.groupby('ad_id') \
                            .agg({'advertiser_id' : 'nunique', 
                                  'campaign_id' : 'nunique', 
                                  'document_id' : 'nunique'}) \
                            .sort_values(['advertiser_id', 'campaign_id', 'document_id'], ascending = False) \
                            .reset_index()
grouped_ad.head(10)

,ad_id,advertiser_id,campaign_id,document_id
0,1,1,1,1
1,2,1,1,1
2,3,1,1,1
3,4,1,1,1
4,5,1,1,1
5,6,1,1,1
6,7,1,1,1
7,8,1,1,1
8,9,1,1,1
9,10,1,1,1


In [81]:
grouped_ad.sum()

ad_id            157625618500
advertiser_id          559583
campaign_id            559583
document_id            559583
dtype: int64

기존에 가정한 것은 이 데이터가 계층구조를 갖고 있다는 것이었다.

즉, 광고주 - 캠페인 - 랜딩페이지 - 광고로 이어지는 계층구조가 있기 때문에

하나의 광고주가 여러 개의 캠페인, 여러 개의 랜딩페이지, 여러 개의 광고를 갖는 것은 가능하고

하나의 광고가 여러 개의 랜딩페이지, 여러 개의 캠페인, 여러 개의 광고주를 갖는 것은 불가능해야 한다.

하지만 EDA를 통해 이 가정이 틀렸음을 확인할 수 있었다.

아래의 코드는 이전에 작성했던 코드들.

groupby를 2번하는 코드.

필요에 따라 위의 코드들과 합친 후 삭제할 예정임.

In [132]:
# 광고주별 캠페인의 기초통계량.
# 광고주는 평균적으로 127개의 캠페인을 갖고 있고, 중위값은 8개이다.

grouped_advertiser.columns = ['advertiser_id', 'the_number_of_campaign'] # 변수명 재정의
grouped_advertiser['the_number_of_campaign'].describe().astype(int)

count     4385
mean       127
std        683
min          1
25%          2
50%          8
75%         36
max      16529
Name: the_number_of_campaign, dtype: int64

In [133]:
# 다시 advertiser_id로 groupby
# 즉, 캠페인을 1개만 갖고 있는 광고주는 704명이다.

re_grouped_advertiser = grouped_advertiser.groupby('the_number_of_campaign').agg({'advertiser_id' : 'count'}).reset_index()
re_grouped_advertiser.columns = ['the_number_of_campaign', 'the_number_of_advertiser']

print(re_grouped_advertiser.head())
print(re_grouped_advertiser.tail())

   the_number_of_campaign  the_number_of_advertiser
0                       1                       704
1                       2                       405
2                       3                       315
3                       4                       245
4                       5                       211
     the_number_of_campaign  the_number_of_advertiser
510                   10046                         1
511                   10552                         1
512                   14844                         1
513                   15450                         1
514                   16529                         1


In [134]:
# 시각화

#ax = sns.boxplot(grouped_advertiser['the_number_of_campaign'])
#ax = sns.distplot(re_grouped_advertiser, kde = True)
#ax.set_yscale('log')

## campaign_id

In [135]:
promoted.head()

,ad_id,document_id,campaign_id,advertiser_id
0,1,6614,1,7
1,2,471467,2,7
2,3,7692,3,7
3,4,471471,2,7
4,5,471472,2,7


In [136]:
# campaign_id로 groupby해서 document_id를 count

grouped_campaign = promoted.groupby('campaign_id').agg({'document_id' : 'count'}).reset_index()
grouped_campaign.head()

,campaign_id,document_id
0,1,31
1,2,57
2,3,2
3,4,267
4,5,16


In [137]:
# 캠페인별 랜딩페이지의 기초통계량
# 캠페인에는 평균적으로 16개의 랜딩페이지가 담겨 있고 중위값은 5개이다.

grouped_campaign.columns = ['campaign_id', 'the_number_of_document'] # 변수명 재정의
grouped_campaign['the_number_of_document'].describe().astype(int)

count    34675
mean        16
std         70
min          1
25%          2
50%          5
75%         12
max       3806
Name: the_number_of_document, dtype: int64

## document_id

In [139]:
promoted.head()

,ad_id,document_id,campaign_id,advertiser_id
0,1,6614,1,7
1,2,471467,2,7
2,3,7692,3,7
3,4,471471,2,7
4,5,471472,2,7


In [140]:
# documente_id로 groupby해서 ad_id를 count

grouped_document = promoted.groupby('document_id').agg({'ad_id' : 'count'}).reset_index()
grouped_document.head()

,document_id,ad_id
0,1,2
1,1305,1
2,1418,1
3,1443,1
4,1456,1


In [142]:
# 랜딩페이지별 기초통계량
# 하나의 랜딩페이지로 연결되는 광고는 평균 3개이고 중위값은 1개이다.

grouped_document.columns = ['document_id', 'the_number_of_ad'] # 변수명 재정의
grouped_document['the_number_of_ad'].describe().astype(int)

count    185709
mean          3
std          44
min           1
25%           1
50%           1
75%           2
max        8662
Name: the_number_of_ad, dtype: int64

In [147]:
grouped_document.head()

,document_id,the_number_of_ad
0,1,2
1,1305,1
2,1418,1
3,1443,1
4,1456,1
